In [1]:
import numpy as np
import scipy.linalg as la
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import folium
from folium.features import DivIcon

In [3]:
from util import *

In [4]:
# import the station information

In [5]:
station_chi = ["古亭","土城","桃園","新竹",
               "二林","新營","小港","美濃",
               "宜蘭","冬山","花蓮","臺東"]

In [6]:
stations_n = ["Guting","Tucheng","Taoyuan","Hsinchu"]
stations_s = ["Erlin","Xinying","Xiaogang","Meinong"]
stations_e = ["Yilan","Dongshan","Hualien","Taitung"]
stations_w = ["Hsinchu","Erlin","Xinying","Xiaogang"]
station_Eng = stations_n+stations_s+stations_e+stations_w

In [7]:
location = [[25.0261,121.5229],[24.9732,121.4441],[25.0015,121.2971],[24.7907,120.9578],
            [23.9363,120.4068],[23.3161,120.2994],[22.5489,120.3567],[22.8871,120.5620],
            [24.7509,121.7588],[24.6129,121.7534],[23.9909,121.6031],[22.7562,121.1120]]

In [8]:
TWN_COORDINATES = (23.5832, 120.5825)
 
# for speed purposes
MAX_RECORDS = 1000

# create empty map zoomed in on San Francisco
Twmap = folium.Map(location=TWN_COORDINATES, tiles='Stamen Terrain', zoom_start=7)


C = location
for i in range(len(C)):
    text = station_Eng[i]
#     folium.Marker(C[i], popup=station_Eng[i]).add_to(Twmap)
    folium.Circle(C[i], 30, fill=True).add_child(folium.Popup(station_Eng[i])).add_to(Twmap)
    folium.Marker(
    [C[i][0], C[i][1]],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12pt;color:blue"><strong>%s<strong></div>' % text,
        )
    ).add_to(Twmap)
    

Twmap.save('Twmap.html') 

display(Twmap)

In [9]:
# import the pollutant data

In [10]:
TW_n = np.zeros((144, 7, len(stations_n)))
TW_s = np.zeros((144, 7, len(stations_s)))
TW_e = np.zeros((144, 7, len(stations_e)))
TW = np.zeros((144, 7, len(stations_n+stations_s+stations_e)))

for i,s in enumerate(stations_n):
    filename = "TW_env/"+s+"_2005_2017_adj.csv"
    data = pd.read_csv(filename, index_col=0)
    TW_n[:,:,i] = data.iloc[:,2:].values

for i,s in enumerate(stations_s):
    filename = "TW_env/"+s+"_2005_2017_adj.csv"
    data = pd.read_csv(filename, index_col=0)
    TW_s[:,:,i] = data.iloc[:,2:].values

for i,s in enumerate(stations_e):
    filename = "TW_env/"+s+"_2005_2017_adj.csv"
    data = pd.read_csv(filename, index_col=0)
    TW_e[:,:,i] = data.iloc[:,2:].values
    
for i,s in enumerate(stations_n+stations_s+stations_e):
    filename = "TW_env/"+s+"_2005_2017_adj.csv"
    data = pd.read_csv(filename, index_col=0)
    TW[:,:,i] = data.iloc[:,2:].values

pollutant = list(data.columns[2:])

In [11]:
def split_ws(X):
    (T, P, S)= X.shape
    W = np.zeros((int(T/2), P, S))
    S = np.zeros((int(T/2), P, S))
    w = 0
    s = 0
    for t in range(T):
        if (t%12<3)|(t%12>8):
            W[w,:,:] = X[t, :,:]
            w += 1
        else:
            S[s,:,:] = X[t, :,:]
            s += 1
    return W, S

In [12]:
T_W, T_S = split_ws(TW)

In [13]:
TW.shape

(144, 7, 12)

In [14]:
# TCCA

In [15]:
# N vs S

In [16]:
inputX, inputY = TW_n, TW_s
inputX, inputY = list(map(lambda x: x-x.mean(axis=0),(inputX, inputY)))
(loading, corr, error) = twoDcca(inputX, inputY, x_regular=0.01, y_regular=0.01, iter_max=30)
for i in range(20):
    (loading_tmp, corr_tmp, error_tmp) = twoDcca(inputX, inputY, x_regular=0.01, y_regular=0.01, iter_max=30)
    if corr[-1]<corr_tmp[-1]:
        corr = corr_tmp
        loading = loading_tmp
print(corr[-1])
corr_n_s = corr[-1]

0.8876952186184383


In [17]:
data = np.r_[loading[1],loading[3]].flatten()
df = pd.DataFrame(data,columns=['coefficient'],index=stations_n+stations_s,dtype=float)
df = df.replace('\$\s+','', regex=True).astype(float).applymap('{:,.3f}'.format)
print(df.T)
print('\n')
print(df.T.to_latex())

            Guting Tucheng Taoyuan Hsinchu  Erlin Xinying Xiaogang Meinong
coefficient  0.051  -0.146  -0.032  -0.988  0.777   0.584   -0.125   0.201


\begin{tabular}{lllllllll}
\toprule
{} & Guting & Tucheng & Taoyuan & Hsinchu &  Erlin & Xinying & Xiaogang & Meinong \\
\midrule
coefficient &  0.051 &  -0.146 &  -0.032 &  -0.988 &  0.777 &   0.584 &   -0.125 &   0.201 \\
\bottomrule
\end{tabular}



In [18]:
data = {'N':loading[0].flatten(),'S':loading[2].flatten()}
df = pd.DataFrame(data,index=pollutant,dtype=float)
df = df.replace('\$\s+','', regex=True).astype(float).applymap('{:,.3f}'.format)
print(df.T)
print('\n')
print(df.T.to_latex())

      SO2      CO      O3    PM10     NOx      NO     NO2
N  -0.001  -0.480   0.322   0.132   0.416  -0.604  -0.333
S  -0.568   0.258  -0.414  -0.067  -0.197   0.619  -0.111


\begin{tabular}{llllllll}
\toprule
{} &     SO2 &      CO &      O3 &    PM10 &     NOx &      NO &     NO2 \\
\midrule
N &  -0.001 &  -0.480 &   0.322 &   0.132 &   0.416 &  -0.604 &  -0.333 \\
S &  -0.568 &   0.258 &  -0.414 &  -0.067 &  -0.197 &   0.619 &  -0.111 \\
\bottomrule
\end{tabular}



In [19]:
# N vs E

In [20]:
inputX, inputY = TW_n, TW_e
inputX, inputY = list(map(lambda x: x-x.mean(axis=0),(inputX, inputY)))
(loading, corr, error) = twoDcca(inputX, inputY, x_regular=0.01, y_regular=0.01, iter_max=30)
for i in range(20):
    (loading_tmp, corr_tmp, error_tmp) = twoDcca(inputX, inputY, x_regular=0.01, y_regular=0.01, iter_max=30)
    if corr[-1]<corr_tmp[-1]:
        corr = corr_tmp
        loading = loading_tmp
print(corr[-1])
corr_n_e = corr[-1]

0.9044489282505639


In [21]:
data = np.r_[loading[1],loading[3]].flatten()
df = pd.DataFrame(data,columns=['coefficient'],index=stations_n+stations_e,dtype=float)
df = df.replace('\$\s+','', regex=True).astype(float).applymap('{:,.3f}'.format)
print(df.T)
print('\n')
print(df.T.to_latex())

             Guting Tucheng Taoyuan Hsinchu  Yilan Dongshan Hualien Taitung
coefficient  -0.627  -0.671  -0.215  -0.331  0.895   -0.051   0.441   0.045


\begin{tabular}{lllllllll}
\toprule
{} &  Guting & Tucheng & Taoyuan & Hsinchu &  Yilan & Dongshan & Hualien & Taitung \\
\midrule
coefficient &  -0.627 &  -0.671 &  -0.215 &  -0.331 &  0.895 &   -0.051 &   0.441 &   0.045 \\
\bottomrule
\end{tabular}



In [22]:
data = {'N':loading[0].flatten(),'E':loading[2].flatten()}
df = pd.DataFrame(data,index=pollutant,dtype=float)
df = df.replace('\$\s+','', regex=True).astype(float).applymap('{:,.3f}'.format)
print(df.T)
print('\n')
print(df.T.to_latex())

      SO2     CO      O3    PM10     NOx      NO     NO2
N  -0.779  0.567   0.170   0.166  -0.051   0.111  -0.015
E   0.027  0.566  -0.402  -0.409  -0.212  -0.010   0.553


\begin{tabular}{llllllll}
\toprule
{} &     SO2 &     CO &      O3 &    PM10 &     NOx &      NO &     NO2 \\
\midrule
N &  -0.779 &  0.567 &   0.170 &   0.166 &  -0.051 &   0.111 &  -0.015 \\
E &   0.027 &  0.566 &  -0.402 &  -0.409 &  -0.212 &  -0.010 &   0.553 \\
\bottomrule
\end{tabular}



In [23]:
# S vs E

In [24]:
inputX, inputY = TW_s, TW_e
inputX, inputY = list(map(lambda x: x-x.mean(axis=0),(inputX, inputY)))
(loading, corr, error) = twoDcca(inputX, inputY, x_regular=0.01, y_regular=0.01, iter_max=30)
for i in range(20):
    (loading_tmp, corr_tmp, error_tmp) = twoDcca(inputX, inputY, x_regular=0.01, y_regular=0.01, iter_max=30)
    if corr[-1]<corr_tmp[-1]:
        corr = corr_tmp
        loading = loading_tmp
print(corr[-1])
corr_s_e = corr[-1]

0.8166533822512857


In [25]:
data = np.r_[loading[1],loading[3]].flatten()
df = pd.DataFrame(data,columns=['coefficient'],index=stations_s+stations_e,dtype=float)
df = df.replace('\$\s+','', regex=True).astype(float).applymap('{:,.3f}'.format)
print(df.T)
print('\n')
print(df.T.to_latex())

              Erlin Xinying Xiaogang Meinong   Yilan Dongshan Hualien Taitung
coefficient  -0.621  -0.166   -0.571  -0.511  -0.499    0.142  -0.297  -0.801


\begin{tabular}{lllllllll}
\toprule
{} &   Erlin & Xinying & Xiaogang & Meinong &   Yilan & Dongshan & Hualien & Taitung \\
\midrule
coefficient &  -0.621 &  -0.166 &   -0.571 &  -0.511 &  -0.499 &    0.142 &  -0.297 &  -0.801 \\
\bottomrule
\end{tabular}



In [26]:
data = {'S':loading[0].flatten(),'E':loading[2].flatten()}
df = pd.DataFrame(data,index=pollutant,dtype=float)
df = df.replace('\$\s+','', regex=True).astype(float).applymap('{:,.3f}'.format)
print(df.T)
print('\n')
print(df.T.to_latex())

      SO2      CO     O3   PM10    NOx      NO     NO2
S  -0.280  -0.713  0.141  0.021  0.357  -0.395  -0.332
E  -0.183  -0.319  0.429  0.078  0.214  -0.033  -0.792


\begin{tabular}{llllllll}
\toprule
{} &     SO2 &      CO &     O3 &   PM10 &    NOx &      NO &     NO2 \\
\midrule
S &  -0.280 &  -0.713 &  0.141 &  0.021 &  0.357 &  -0.395 &  -0.332 \\
E &  -0.183 &  -0.319 &  0.429 &  0.078 &  0.214 &  -0.033 &  -0.792 \\
\bottomrule
\end{tabular}



In [27]:
data = [corr_n_s, corr_s_e, corr_n_e]
df = pd.DataFrame(data,columns=['Correlation'],index=['N vs S','S vs E','N vs E'],dtype=float)
df = df.replace('\$\s+','', regex=True).astype(float).applymap('{:,.3f}'.format)
print(df)
print('\n')
print(df.T.to_latex())

       Correlation
N vs S       0.888
S vs E       0.817
N vs E       0.904


\begin{tabular}{llll}
\toprule
{} & N vs S & S vs E & N vs E \\
\midrule
Correlation &  0.888 &  0.817 &  0.904 \\
\bottomrule
\end{tabular}



In [28]:
# split data by summer and winter

In [29]:
N_W, N_S = split_ws(TW_n)
S_W, S_S = split_ws(TW_s)
E_W, E_S = split_ws(TW_e)

In [30]:
# N vs S

In [31]:
inputX, inputY = N_W, S_W
inputX, inputY = list(map(lambda x: x-x.mean(axis=0),(inputX, inputY)))
(loading, corr, error) = twoDcca(inputX, inputY, x_regular=0.01, y_regular=0.01, iter_max=40)
for i in range(30):
    (loading_tmp, corr_tmp, error_tmp) = twoDcca(inputX, inputY, x_regular=0.01, y_regular=0.01, iter_max=100)
    if corr[-1]<corr_tmp[-1]:
        corr = corr_tmp
        loading = loading_tmp
print(corr[-1])
loading_ww = loading
corr_n_s_ww = corr[-1]

0.9402669845420455


In [32]:
inputX, inputY = N_S, S_S
inputX, inputY = list(map(lambda x: x-x.mean(axis=0),(inputX, inputY)))
(loading, corr, error) = twoDcca(inputX, inputY, x_regular=0.01, y_regular=0.01, iter_max=40)
for i in range(30):
    (loading_tmp, corr_tmp, error_tmp) = twoDcca(inputX, inputY, x_regular=0.01, y_regular=0.01, iter_max=40)
    if corr[-1]<corr_tmp[-1]:
        corr = corr_tmp
        loading = loading_tmp
print(corr[-1])
loading_ss = loading
corr_n_s_ss = corr[-1]

0.8891442854167241


In [33]:
data = {"Winter":np.r_[loading_ww[1],loading_ww[3]].flatten(),"Summer":np.r_[loading_ss[1],loading_ss[3]].flatten()}
df = pd.DataFrame(data,index=stations_n+stations_s,dtype=float)
df = df.replace('\$\s+','', regex=True).astype(float).applymap('{:,.3f}'.format)
print(df.T)
print('\n')
print(df.T.to_latex())

        Guting Tucheng Taoyuan Hsinchu   Erlin Xinying Xiaogang Meinong
Winter   0.423   0.061   0.078   0.901  -0.925  -0.345    0.161  -0.024
Summer  -0.401   0.277   0.188   0.853  -0.832  -0.438   -0.205  -0.271


\begin{tabular}{lllllllll}
\toprule
{} &  Guting & Tucheng & Taoyuan & Hsinchu &   Erlin & Xinying & Xiaogang & Meinong \\
\midrule
Winter &   0.423 &   0.061 &   0.078 &   0.901 &  -0.925 &  -0.345 &    0.161 &  -0.024 \\
Summer &  -0.401 &   0.277 &   0.188 &   0.853 &  -0.832 &  -0.438 &   -0.205 &  -0.271 \\
\bottomrule
\end{tabular}



In [34]:
data = {'N(Winter)':loading_ww[0].flatten(),'S(Winter)':loading_ww[2].flatten(),'N(Summer)':loading_ss[0].flatten(),'S(Summer)':loading_ss[2].flatten()}
df = pd.DataFrame(data,index=pollutant,dtype=float)
df = df.replace('\$\s+','', regex=True).astype(float).applymap('{:,.3f}'.format)
print(df.T)
print('\n')
print(df.T.to_latex())

             SO2      CO      O3    PM10     NOx      NO     NO2
N(Winter)  0.122   0.672  -0.176  -0.084   0.298  -0.083  -0.633
S(Winter)  0.229   0.805   0.357   0.071   0.291  -0.242   0.153
N(Summer)  0.541   0.347  -0.534  -0.060  -0.260   0.475  -0.072
S(Summer)  0.002  -0.134   0.359  -0.003  -0.232  -0.740   0.501


\begin{tabular}{llllllll}
\toprule
{} &    SO2 &      CO &      O3 &    PM10 &     NOx &      NO &     NO2 \\
\midrule
N(Winter) &  0.122 &   0.672 &  -0.176 &  -0.084 &   0.298 &  -0.083 &  -0.633 \\
S(Winter) &  0.229 &   0.805 &   0.357 &   0.071 &   0.291 &  -0.242 &   0.153 \\
N(Summer) &  0.541 &   0.347 &  -0.534 &  -0.060 &  -0.260 &   0.475 &  -0.072 \\
S(Summer) &  0.002 &  -0.134 &   0.359 &  -0.003 &  -0.232 &  -0.740 &   0.501 \\
\bottomrule
\end{tabular}



In [35]:
# N vs E

In [36]:
inputX, inputY = N_W, E_W
inputX, inputY = list(map(lambda x: x-x.mean(axis=0),(inputX, inputY)))
(loading, corr, error) = twoDcca(inputX, inputY, x_regular=0.01, y_regular=0.01, iter_max=30)
for i in range(20):
    (loading_tmp, corr_tmp, error_tmp) = twoDcca(inputX, inputY, x_regular=0.01, y_regular=0.01, iter_max=30)
    if corr[-1]<corr_tmp[-1]:
        corr = corr_tmp
        loading = loading_tmp
print(corr[-1])
loading_ww = loading
corr_ww = corr[-1]
corr_n_e_ww = corr[-1]

0.9302647344938701


In [37]:
inputX, inputY = N_S, E_S
inputX, inputY = list(map(lambda x: x-x.mean(axis=0),(inputX, inputY)))
(loading, corr, error) = twoDcca(inputX, inputY, x_regular=0.01, y_regular=0.01, iter_max=30)
for i in range(20):
    (loading_tmp, corr_tmp, error_tmp) = twoDcca(inputX, inputY, x_regular=0.01, y_regular=0.01, iter_max=30)
    if corr[-1]<corr_tmp[-1]:
        corr = corr_tmp
        loading = loading_tmp
print(corr[-1])
loading_ss = loading
corr_ss = corr[-1]
corr_n_e_ss = corr[-1]

0.9143104446833404


In [38]:
data = {"Winter":np.r_[loading_ww[1],loading_ww[3]].flatten(),"Summer":np.r_[loading_ss[1],loading_ss[3]].flatten()}
df = pd.DataFrame(data,index=stations_n+stations_e,dtype=float)
df = df.replace('\$\s+','', regex=True).astype(float).applymap('{:,.3f}'.format)
print(df.T)
print('\n')
print(df.T.to_latex())

        Guting Tucheng Taoyuan Hsinchu   Yilan Dongshan Hualien Taitung
Winter   0.606   0.704   0.366   0.058  -0.805   -0.308  -0.503  -0.070
Summer  -0.419  -0.225  -0.783  -0.401   0.843    0.195   0.464   0.188


\begin{tabular}{lllllllll}
\toprule
{} &  Guting & Tucheng & Taoyuan & Hsinchu &   Yilan & Dongshan & Hualien & Taitung \\
\midrule
Winter &   0.606 &   0.704 &   0.366 &   0.058 &  -0.805 &   -0.308 &  -0.503 &  -0.070 \\
Summer &  -0.419 &  -0.225 &  -0.783 &  -0.401 &   0.843 &    0.195 &   0.464 &   0.188 \\
\bottomrule
\end{tabular}



In [39]:
data = {'N(Winter)':loading_ww[0].flatten(),'E(Winter)':loading_ww[2].flatten(),'N(Summer)':loading_ss[0].flatten(),'E(Summer)':loading_ss[2].flatten()}
df = pd.DataFrame(data,index=pollutant,dtype=float)
df = df.replace('\$\s+','', regex=True).astype(float).applymap('{:,.3f}'.format)
print(df.T)
print('\n')
print(df.T.to_latex())

              SO2      CO      O3    PM10     NOx      NO     NO2
N(Winter)  -0.458  -0.333   0.248   0.072  -0.401   0.433   0.514
E(Winter)   0.597   0.254  -0.324  -0.139  -0.402   0.461   0.285
N(Summer)  -0.111   0.113   0.074   0.188   0.450  -0.283  -0.807
E(Summer)  -0.272   0.371  -0.127  -0.209  -0.099  -0.473   0.704


\begin{tabular}{llllllll}
\toprule
{} &     SO2 &      CO &      O3 &    PM10 &     NOx &      NO &     NO2 \\
\midrule
N(Winter) &  -0.458 &  -0.333 &   0.248 &   0.072 &  -0.401 &   0.433 &   0.514 \\
E(Winter) &   0.597 &   0.254 &  -0.324 &  -0.139 &  -0.402 &   0.461 &   0.285 \\
N(Summer) &  -0.111 &   0.113 &   0.074 &   0.188 &   0.450 &  -0.283 &  -0.807 \\
E(Summer) &  -0.272 &   0.371 &  -0.127 &  -0.209 &  -0.099 &  -0.473 &   0.704 \\
\bottomrule
\end{tabular}



In [40]:
# S vs E

In [41]:
inputX, inputY = S_W, E_W
inputX, inputY = list(map(lambda x: x-x.mean(axis=0),(inputX, inputY)))
(loading, corr, error) = twoDcca(inputX, inputY, x_regular=0.01, y_regular=0.01, iter_max=30)
for i in range(20):
    (loading_tmp, corr_tmp, error_tmp) = twoDcca(inputX, inputY, x_regular=0.01, y_regular=0.01, iter_max=30)
    if corr[-1]<corr_tmp[-1]:
        corr = corr_tmp
        loading = loading_tmp
print(corr[-1])
loading_ww = loading
corr_s_e_ww = corr[-1]

0.9037724261116833


In [42]:
inputX, inputY = S_S, E_S
inputX, inputY = list(map(lambda x: x-x.mean(axis=0),(inputX, inputY)))
(loading, corr, error) = twoDcca(inputX, inputY, x_regular=0.01, y_regular=0.01, iter_max=30)
for i in range(20):
    (loading_tmp, corr_tmp, error_tmp) = twoDcca(inputX, inputY, x_regular=0.01, y_regular=0.01, iter_max=30)
    if corr[-1]<corr_tmp[-1]:
        corr = corr_tmp
        loading = loading_tmp
print(corr[-1])
loading_ss = loading
corr_s_e_ss = corr[-1]

0.8211501266448208


In [43]:
data = {"Winter":np.r_[loading_ww[1],loading_ww[3]].flatten(),"Summer":np.r_[loading_ss[1],loading_ss[3]].flatten()}
df = pd.DataFrame(data,index=stations_s+stations_e,dtype=float)
df = df.replace('\$\s+','', regex=True).astype(float).applymap('{:,.3f}'.format)
print(df.T)
print('\n')
print(df.T.to_latex())

         Erlin Xinying Xiaogang Meinong   Yilan Dongshan Hualien Taitung
Winter   0.962  -0.058    0.113   0.240   0.462    0.154   0.751   0.445
Summer  -0.354  -0.810   -0.305  -0.354  -0.747    0.349  -0.321  -0.465


\begin{tabular}{lllllllll}
\toprule
{} &   Erlin & Xinying & Xiaogang & Meinong &   Yilan & Dongshan & Hualien & Taitung \\
\midrule
Winter &   0.962 &  -0.058 &    0.113 &   0.240 &   0.462 &    0.154 &   0.751 &   0.445 \\
Summer &  -0.354 &  -0.810 &   -0.305 &  -0.354 &  -0.747 &    0.349 &  -0.321 &  -0.465 \\
\bottomrule
\end{tabular}



In [44]:
data = {'S(Winter)':loading_ww[0].flatten(),'E(Winter)':loading_ww[2].flatten(),'S(Summer)':loading_ss[0].flatten(),'E(Summer)':loading_ss[2].flatten()}
df = pd.DataFrame(data,index=pollutant,dtype=float)
df = df.replace('\$\s+','', regex=True).astype(float).applymap('{:,.3f}'.format)
print(df.T)
print('\n')
print(df.T.to_latex())

              SO2      CO      O3    PM10     NOx      NO     NO2
S(Winter)   0.396   0.024  -0.263  -0.019  -0.631   0.392   0.470
E(Winter)   0.298   0.525  -0.168  -0.076  -0.285   0.712   0.117
S(Summer)  -0.622  -0.163   0.182   0.010   0.440  -0.279  -0.532
E(Summer)  -0.088   0.024   0.265   0.048   0.262   0.616  -0.686


\begin{tabular}{llllllll}
\toprule
{} &     SO2 &      CO &      O3 &    PM10 &     NOx &      NO &     NO2 \\
\midrule
S(Winter) &   0.396 &   0.024 &  -0.263 &  -0.019 &  -0.631 &   0.392 &   0.470 \\
E(Winter) &   0.298 &   0.525 &  -0.168 &  -0.076 &  -0.285 &   0.712 &   0.117 \\
S(Summer) &  -0.622 &  -0.163 &   0.182 &   0.010 &   0.440 &  -0.279 &  -0.532 \\
E(Summer) &  -0.088 &   0.024 &   0.265 &   0.048 &   0.262 &   0.616 &  -0.686 \\
\bottomrule
\end{tabular}



In [46]:
# summary

In [45]:
data = {'N vs S':[corr_n_s_ww, corr_n_s_ss], "S vs E":[corr_s_e_ww, corr_s_e_ss], "N vs E":[corr_n_e_ww, corr_n_e_ss]}
df = pd.DataFrame(data,index=['Winter','Summer'],dtype=float)
df = df.replace('\$\s+','', regex=True).astype(float).applymap('{:,.3f}'.format)
print(df)
print('\n')
print(df.to_latex())

       N vs S S vs E N vs E
Winter  0.940  0.904  0.930
Summer  0.889  0.821  0.914


\begin{tabular}{llll}
\toprule
{} & N vs S & S vs E & N vs E \\
\midrule
Winter &  0.940 &  0.904 &  0.930 \\
Summer &  0.889 &  0.821 &  0.914 \\
\bottomrule
\end{tabular}

